In [1]:
from datum import *

In [ ]:
# irreps = '24x1e'
e3nn.IrrepsArray('24x1e',irreps_array)

In [66]:
import e3nn_jax as e3nn

In [69]:
import jax
from einops import rearrange

In [40]:
print(get_nucleotide_index('DA'))
print(get_nucleotide_index('DT'))
print(get_nucleotide_index('DG'))
print(get_nucleotide_index('DC'))
print(get_nucleotide_index('DI'))
print(get_nucleotide_index('DU'))
print(get_nucleotide_index('UNK'))


8
12
10
9
11
13
1


In [82]:

pdb_ids = ["1JMC","1ZEW","5CRX","1BNA","3BEP","1HRY","3PX6","1BNA","2JYK","6NY1"] # last one has protein in it
#last one has RNAs
a = []
###Dictionary of res = array of tuples (atom token, atom coords)
residues = {8 :[],
            12 :[],
            10 :[],
             9 :[],
            11 :[],
            13 :[]}
for id in pdb_ids:
    nuc_dat = NucleicDatum.fetch_pdb_id(id)
    for i in range(len(nuc_dat.nuc_token)):
        # res_center_coord = datum.atom_coord[..., mask_center, :] #shape 12,1,3
        # relative_vectors = all_atom_coord - res_center_coord[:, np.newaxis,:] #shape 12,24,3

        tok = nuc_dat.nuc_token[i] 
        if tok == 1 or tok==0 or tok==5 or tok ==6 or tok==3 or tok==2:
            continue
        atom_coords = nuc_dat.atom_coord[i]
        atom_token = nuc_dat.atom_token[i] 
        center_atom_token = atom_index("C3'")
        mask_center = atom_token == center_atom_token # True for location of C3
        atom_mask = nuc_dat.atom_mask[i] 
        true_index = None
        for j, value in enumerate(mask_center):
            if value:
                true_index = j
                break
        coords = nuc_dat.atom_coord[i]
        res_center_coord = coords[true_index]
        coords_centered = coords - res_center_coord
        coords_centered_flat = rearrange(coords_centered, 'h c->(h c)')
        # print(coords_centered_flat.shape)
        vecs = e3nn.IrrepsArray('24x1e',coords_centered_flat)
        one_hot = np.zeros(14)
        one_hot[tok] = 1
        scalars = e3nn.IrrepsArray('14x0e', one_hot) #runs into weird GPU error

        irreps = e3nn.concatenate([scalars, vecs])
        a.append(irreps)
        
        # residues[tok].append(coords_centered)

        

In [83]:
len(a)

310

In [84]:
import pickle

In [88]:
with open('moleculib/nucleic/DNA_res_irreps.pkl', 'wb') as f:
    pickle.dump(a, f)

print("List of irreps saved to DNA_res_irreps.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'moleculib/nucleic/DNA_res_irreps.pkl'